# Forgather

A notebook for experimenting with Forgather's syntax.

In [ ]:
import sys, os
modules_path = os.path.join('..', 'src')
if modules_path not in sys.path: sys.path.insert(0, modules_path)

from pprint import pp, pformat

from IPython import display as ds

from forgather import Latent
from forgather.config import ConfigEnvironment
from forgather.preprocess import PPEnvironment
from aiws.construct import generate_code

# Render code via Markdown render
def render_codeblock(language, source, header=None):
    if header is None:
        header = ""
    else:
        header = header + '\n'
    display(ds.Markdown(f"{header}```{language}\n{source}\n\n```"))

# Show common syntax definition.
with open(os.path.join('..', 'syntax.md'), 'r') as f:
    display(ds.Markdown(f.read()))

---
## Create Config Environment

```python
ConfigEnvironment(
    searchpath: Iterable[str | os.PathLike] | str | os.PathLike = tuple("."),
    pp_environment: Environment = None,
    global_vars: Dict[str, Any] = None,
):
```

- searchpath: A list of directories to search for templates in.
- pp_environment: Override the default Jinja2 environment class with another implementation.
- global_vars: Jinja2 global variables visible to all templates.

In [ ]:
env = ConfigEnvironment()

## Define Input Document

In [ ]:
document = """
# Compute powers-of-two in a list, returning a list.
!singleton:list
    - !singleton:map
        - !lambda:pow [ !var "arg0", 2 ]
        - [ 1, 2, 3, 4 ]
"""

## Convert Document to Graph

```python
class ConfigEnvironment:
... 
    def load(
        self,
        config_path: os.PathLike | str,
        /,
        **kwargs,
    ) -> Config:
...
    def load_from_string(
        self,
        config: str,
        /,
        **kwargs,
    ) -> Config:
```

- load: Load a template from a path; all paths relative to 'searchpaths' are searched for the template.
    - config_path: The relative (to searchpaths) template path.
    - kwargs: These are passed into the context of the template.
- load_from_string: As with load, but a Python string defines the template body; Note that this bypasses the template loader.
    - config: A Python string with a Jinja2 template.
    - kwargs: Passed to the template.

In [ ]:
graph = env.load_from_string(document).config
render_codeblock("python", pformat(graph), "### Node Graph")

## Convert Graph to YAML

Convert the node-graph to a YAML representation. This may not be exactly the same as it was in the source template, but should be symantically equivalent.

In [ ]:
render_codeblock("yaml", Latent.to_yaml(graph))

## Materialize Graph

Construct the objects in the graph.

In [ ]:
obj = Latent.materialize(graph)
render_codeblock("python", pformat(obj), "### Objects")

## Convert Graph to Python

Convert the graph into the equivalent Python code.

The output is a dictionary containing the following:

- imports: A list of tuples describing the required imports.
- dynamic_imports: A list of tuples describing the required dynamic imports.
- variables: A list of tuples describing all of the variables.
- main_body: The main-body of the generated Python code.

Note: Forgather does not use the intermediary step of converting the graph to code as part of Latent.materialize(); the normal execution path directly interprets the node-graph when constructing objects.

In [ ]:
generated_code = Latent.to_py(graph)

def render_to_py(generated_code):
    # Normal imports
    # 
    if len(generated_code['imports']):
        print("imports: list[tuple[module: str, symbol_name: str]]")
        pp(generated_code['imports'])

    # The dynamic imports (imports, where a python file is specified)
    # list[tuple[module: str, symbol: str, searchpath: list[str]]]
    if len(generated_code['dynamic_imports']):
        print("\ndynamic-imports: list[tuple[module: str, symbol_name: str, searchpath: list[str]]]")
        pp(generated_code['dynamic_imports'])

    # Variable substitutions
    if len(generated_code['variables']):
        print("\nvariables: list[tuple[name: str, is_undefined: bool, default: Any]]")
        pp(generated_code['variables'])
    
    render_codeblock("python", generated_code['main_body'], "#### Main Body:")

render_to_py(generated_code)

## Convert Graph to Python with Jinja2 Template

This function takes the output from Latent.to_py(graph) and uses it to render Pyhon code using a Jinja2 template. If the template is unspecified, an implicit "built-in" template is used, which will generate appropriate import and dynamic import statements, where required.

```python
def generate_code(
    obj,
    template_name: Optional[str] = None,
    template_str: Optional[str] = None,
    searchpath: Optional[List[str | os.PathLike] | str | os.PathLike] = ".",
    env=None,  # jinja2 environment or compatible API
    output_file: Optional[str | os.PathLike] = None,
    return_value: Optional[Any] = Undefined,
    **kwargs,
) -> Any:
```

See 'help(generate_code)' for details.

In [ ]:
generated_code = generate_code(graph)
render_codeblock("python", generated_code, "### Generated Code", )

## Execute Generated Code

Execute the generated code, then call the generated 'construct' function to construct the object.

Note: Lambda nodes with args are not working at present (although Latent.materialize() works)

In [ ]:
exec(generated_code)
obj = construct()
render_codeblock("python", obj, "### Construct Object", )